In [1]:
from src.opensource.llama_frontend import chat_pipeline, get_clues
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from src.opensource.templates import SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE, SYS_BLOOMZ_TEMPLATE, INST_BLOOMZ_TEMPLATE

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [3]:
inst_template = PromptTemplate.from_template(INST_LLAMA_TEMPLATE)
inst_template = inst_template.format(state='Jharkhand')
print(inst_template)


    [INST] Name the object on the basis of the above clues from Jharkhand. After your guess, I will let you know if you are correct or not. If you're correct, do not make any further guesses and end the conversation. If you are wrong, make your second and final guess. I do not need to know your reasoning behind the answer. Just tell me the answer and nothing else. If you do not know the answer, say that you do not know the answer. Format your answer in the form of ANSWER: your_answer_here.[/INST]



In [4]:
df_jharkhand = pd.read_csv('/home/t-sahuja/cultural_artifacts/clues/jharkhand/jharkhand_clues_1.csv')

In [5]:
df_jharkhand.iloc[3]['clues'].split('\n')

['It is a musical intrument played with both hands.',
 'It is integral to the tribal dance and hold importance for the tribes of Jharkhand',
 '']

In [6]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [7]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row['clues'].strip().split('\n')
        artifact = row['artifact'].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue}\n"
            fin_clues = output.strip()
        
        print(fin_clues, "clues")
        agent = chat_pipeline(clue_list = fin_clues, prompt_text=SYS_LLAMA_TEMPLATE, conversation_buffer=conversation_buffer)
        guess1 = agent.predict(human_input=inst_template)
        guess1 = guess1.split(':')[1].strip().lower()
        print(guess1, "Guess1")
        if artifact in guess1:
            df_eval = df_eval.append({'guess1': guess1, 'guess2': 'NA', 'ground_truth': artifact, 'clues': row['clues']}, ignore_index=True)
            conversation_buffer.clear()
            continue
        else:
            guess2 = agent.predict(human_input="Your first guess is not correct")
            guess2 = guess2.split(':')[1].strip().lower()
            print(guess2, "Guess2")
            df_eval = df_eval.append({'guess1': guess1, 'guess2': guess2, 'ground_truth': artifact, 'clues': row['clues']}, ignore_index=True)
            conversation_buffer.clear()
    return df_eval
            

In [8]:
conversation_buffer.clear()

In [9]:
df_eval = get_outputs(df_jharkhand, df_eval)

artifact--0---
CLUE-1: It is a National Park.
CLUE-2: It is located in Ormanjhi
CLUE-3: It is named after a famous freedom fighter.
CLUE-4: It was a tribute to that freedom fighter clues
birsa munda national park Guess1
artifact--1---
CLUE-1: Waterfall located in Ranchi district
CLUE-2: It is surrounded by both forest and hills.
CLUE-3: There is a Buddha temple there. clues
the social artifact i am guessing is the "rock garden" located in ranchi, jharkhand. Guess1
my second and final guess is the "hundru falls" located in ranchi, jharkhand. Guess2
artifact--2---
CLUE-1: Highest fall of Jharkhand.
CLUE-2: It is a common weekend picnic and tourist spot. clues
ranchi hill. Guess1
hazaribagh national park Guess2
artifact--3---
CLUE-1: It is a musical intrument played with both hands.
CLUE-2: It is integral to the tribal dance and hold importance for the tribes of Jharkhand clues
dhol. Guess1
artifact--4---
CLUE-1: It is a artform done on wood by the Tribes of Jharkhand
CLUE-2: It is at dis

In [10]:
df_eval.to_csv('jharkhand_evals_with_state_name.csv', index=False)

In [11]:
df_eval

,guess1,guess2,ground_truth,clues
0,birsa munda national park,NA,birsa munda national park,It is a National Park.\nIt is located in Orman...
1,"the social artifact i am guessing is the ""rock...","my second and final guess is the ""hundru falls...",jonha falls,Waterfall located in Ranchi district\nIt is su...
2,ranchi hill.,hazaribagh national park,hunduru falls,Highest fall of Jharkhand.\nIt is a common wee...
3,dhol.,NA,dhol,It is a musical intrument played with both han...
4,madhubani painting,handicrafts,tribal woodwork,It is a artform done on wood by the Tribes of ...
5,ranchi,jamshedpur,netrahat,It is a hill station for weekend picinic spot\...
6,the object in jharkhand is the jharkhand war m...,the object in jharkhand is the jawaharlal nehr...,juliee park,It is very famous park in Jameshdpur\nIt is a ...
7,baidyanath dham,kalesar,baba baidyanath temple,It is a very famous Lord Shiva temple at Devga...
8,bandhavgarh national park,kaziranga national park,betla national park,"National Park and it is famous for elephant, t..."
9,santhal painting,madhubani painting,paitkar paintings,A famous painting from a parrticular community...


Bad pipe message: %s [b'4\xb0I\xa98_Y2\xbc\xc8\xdc#-\x813\x82s\x8e K\x9c\xe4=P\x8f\xbc\xf9\xd0.\xd9z\xb9\xf4cyn\xb7\x18\x08\x88\x84S\xba4t\xd7\xe8&\xf4\x12\xdc\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'aQ\xfd\x83\xe1\x08\xbd\xa6v+\x04\xbb\xbc\xf0gpL\x03\x00\x00|\xc0,']
Bad pipe message: %s [b"\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/